In [1]:
import tensorflow as tf 
import tensorflow.keras.layers as layers
import tensorflow.keras.models as model

# BASNET ATTEMPT 1


**Basic Architecture of BASNet consist of two Module, Prediction Module and Refined Module**

- lets start with prediction Module
- Construct helper function 
- The Prediction module is U-Net like structure (encoder-decoder)
- For Encoder we are going to use residual blocks



# Helper Function

In [2]:
def Residual_block(x,filters,stride=1,downsample=None,no_relu=None):
    '''
    Creates residual block with two convo2D with 3*3 kernal size
    '''
    residual = x
    # convblock1
    l = layers.Conv2D(kernel_size=(3,3), filters=filters, strides=stride, padding="same", use_bias=False)(x)
    l = layers.BatchNormalization()(l)
    l = layers.ReLU()(l)
    # convblock1
    l = layers.Conv2D(kernel_size=(3,3), filters=filters, strides=1, padding="same", use_bias=False)(l)
    l = layers.BatchNormalization()(l)
    
    if downsample is not None:
        residual = downsample
    
    l = layers.add()([l,residualdual])
    if not no_relu:
        l = layers.Activation('relu')(l)
    return l
    
    

In [5]:
def make_layer(x,block,in_filter,filters, blocks_num, stride=1, expansion=1):
    """
    apply multiple residual blocks.
    x_in: input tensor
    block: block to apply
    in_filter: input tensor channels
    filters: output tensor channels
    blocks_num: number of time block to be applied
    stride: stride
    expansion: expand last dimension
    """
    downsample = None
    if stride!=1 or in_filter != filters*expansion:
        downsample = layers.Conv2D(((filters*expansion)),kernel_size=(1,1),strides=stride,use_bias=False)(x)
        downsample = layers.BatchNormalization()(downsample)
        downsample = layers.Activation("relu")(downsample)
    l = block(x,filters,stride,downsample)
    for i in range(1,blocks_num):
        if i == (blocks_num - 1):
            l = block(x,filters,stride=1,no_relu=True)
        else:
            l = block(x,filters,stride=1,no_relu=False)
    return l

In [8]:
def conv_bn_relu(x, filters, dilation=1, stride=1, padding="same"):
    """
    apply Convolution+BN+Relu layer
    """
    l = layers.Conv2D(filters=filters, kernel_size=(3, 3), strides=stride, padding=padding, dilation_rate=dilation)(x)
    l = layers.BatchNormalization()(l)
    l = layers.Activation("relu")(l)
    return l

In [9]:
def conv_block(x, filters, dilation=1):
    """
    Creates a convolution block with three consecutive 3*3 conv's
    """
    l = conv_bn_relu(x=x_in, filters=filters, dilation=dilation)
    l = conv_bn_relu(x=x, filters=filters, dilation=dilation)
    l = conv_bn_relu(x=x, filters=filters, dilation=dilation)

    return l

In [10]:
def resize_bilinear(x, factor=2):
    """
    Resize input tensor using bilinear upsampling strategy
    factor: resize input with given factor
    """
    h = tf.keras.backend.int_shape(x)[1] * factor
    w = tf.keras.backend.int_shape(x)[2] * factor
    return tf.image.resize(x, [h, w])

In [12]:
def segmentation_head(x, filters, final_size):
    """
    Segmentation head uses convolution to map each stage output to final 
    number of classes, if final size is not null then also resizes to given size.
    x_in: input tensor
    out_planes: # output classes
    final_size: size to which each stage output will be resized 
    """
    l = layers.Conv2D(filters, kernel_size=(3, 3), strides=1, padding="same",)(x)

    if final_size is not None:
        l = tf.image.resize(l, size=final_size)

    return l